# 강의 11주차: midm-food-order-understanding

1. KT-AI/midm-bitext-S-7B-inst-v1 를 주문 문장 이해에 미세 튜닝

- food-order-understanding-small-3200.json (학습)
- food-order-understanding-small-800.json (검증)


종속적인 필요 내용
- huggingface 계정 설정 및 llama-2 사용 승인
- 로깅을 위한 wandb


history

v1.2
- KT-AI/midm-bitext-S-7B-inst-v1 에 safetensors 포맷이 올라왔기에, 해당 리포에서 받도록 설정 변경
- 전체 과정 재검증

In [ ]:
pip install transformers peft accelerate optimum bitsandbytes trl wandb einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional
import re

import torch
import tyro
from accelerate import Accelerator
from datasets import load_dataset, Dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)

from trl import SFTTrainer

from trl.trainer import ConstantLengthDataset

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
my_dataset=load_dataset("nsmc")#nsmc데이터셋 확인
my_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

드라이브 마운트 후 파일 업로드
- food-order-understanding-small-3200.json
- food-order-understanding-small-800.json

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# 내가 업로드한 경로 기억 해둠

# /gdrive/MyDrive/Lectures/2023/nlp/food-order-understanding-small-3200.json
# /gdrive/MyDrive/Lectures/2023/nlp/food-order-understanding-small-800.json

# 매개 변수 설정

In [ ]:
@dataclass
class ScriptArguments:
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "the cache dir"}
    )
    model_name: Optional[str] = field(
        default="meta-llama/Llama-2-7b-chat-hf", metadata={"help": "the model name"}
    )

    dataset_name: Optional[str] = field(
        default="nsmc",#dataset의 이름을 None에서 nsmc로 수정
        metadata={"help": "the dataset name"},
    )
    seq_length: Optional[int] = field(
        default=1024, metadata={"help": "the sequence length"}
    )
    num_workers: Optional[int] = field(
        default=8, metadata={"help": "the number of workers"}
    )
    training_args: TrainingArguments = field(
        default_factory=lambda: TrainingArguments(
            output_dir="./results",
            # max_steps=500,
            logging_steps=20,
            # save_steps=10,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=2,
            gradient_checkpointing=False,
            group_by_length=False,
            learning_rate=1e-4,
            lr_scheduler_type="cosine",
            # warmup_steps=100,
            warmup_ratio=0.03,
            max_grad_norm=0.3,
            weight_decay=0.05,
            save_total_limit=20,
            save_strategy="epoch",
            num_train_epochs=1,
            optim="paged_adamw_32bit",
            fp16=True,
            remove_unused_columns=False,
            report_to="wandb",
        )
    )

    packing: Optional[bool] = field(
        default=True, metadata={"help": "whether to use packing for SFTTrainer"}
    )

    peft_config: LoraConfig = field(
        default_factory=lambda: LoraConfig(
            r=8,
            lora_alpha=16,
            lora_dropout=0.05,
            target_modules=["c_attn", "c_proj", "c_fc"],
            bias="none",
            task_type="CAUSAL_LM",
        )
    )

    merge_with_final_checkpoint: Optional[bool] = field(
        default=False, metadata={"help": "Do only merge with final checkpoint"}
    )

# 유틸리티

In [ ]:
def chars_token_ratio(dataset, tokenizer, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

#### -chars_token_ratio와 print_trainable_parameters는 helper함수이다.(토큰당 캐릭터수를 알려주고 parameter들을 보여주는)

# 데이터 로딩

### prepare_sample_text함수
##### nsmc데이터셋의 feature특징은 'id':string,'document':string,'label':class label타입이므로 그에 따라 0은 부정,1은 긍정의 값을 내도록 수정했다.

In [ ]:
def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset."""

    prompt_template = """###System;{System}
    \n###User;{User}
    \n###Midm;{Midm}"""

    default_system_msg = (
       "너는 먼저 사용자가 작성한 리뷰의 긍정 또는 부정여부를 판단해야 한다. 이로부터 그에 대한 결과를 '긍정' 또는 '부정' 으로 출력해야한다."
    )

    if isinstance(example, dict):
        # Assuming 'document' and 'label' are present in the dictionary
        label = int(example["label"])
        text = (
            prompt_template.format(System=default_system_msg,
                                   User=example["document"],
                                   Midm="긍정" if label == 1 else "부정")
        )
    elif isinstance(example, str):
        # Handle the case where example is a string
        text = prompt_template.format(System=default_system_msg, User=example, Midm="Unknown")
    else:
        print(f"Unsupported example format: {type(example)}")
        raise ValueError("Unsupported example format")

    return text


In [ ]:
train_data = my_dataset["train"]#그냥 prepare_sample이 어떤 형식인지 보려고 내가 짠거-수행하지 말아주세요(참고만)
num_samples_to_test = 10
for i in range(num_samples_to_test):
    example = train_data[i]
    processed_text = prepare_sample_text(example)
    print(f"예제:{i + 1}:\n{processed_text}\n{'='*50}")


Processed Text for Example 1:
###System;너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다.
    
###User;아 더빙.. 진짜 짜증나네요 목소리
    
###Midm;부정
Processed Text for Example 2:
###System;너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다.
    
###User;흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
    
###Midm;긍정
Processed Text for Example 3:
###System;너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다.
    
###User;너무재밓었다그래서보는것을추천한다
    
###Midm;부정
Processed Text for Example 4:
###System;너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다.
    
###User;교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
    
###Midm;부정
Processed Text for Example 5:
###System;너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다.
    
###User;사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
    
###Midm;긍정
Processed Text for Example 6:
###System;너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다.
    
###User;막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
    
###Midm;부정
Processed Text for Example 7:
###System;너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다.
    
###User;원작의 긴장감을 제대로 살려내지못했다.
    
###Midm;부정
Processed Text for Example 8:
###System;

In [ ]:
def create_datasets(tokenizer, args):
    dataset = load_dataset(args.dataset_name, cache_dir=args.cache_dir)
    train_data = dataset['train'].select(range(2100))
    valid_data = dataset['test'].select(range(1000))
    chars_per_token = chars_token_ratio(train_data, tokenizer)
    print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")
    train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=args.seq_length,
        chars_per_token=chars_per_token,
    )
    # valid_dataset = load_dataset(
    #     script_args.dataset_name,
    #     cache_dir=script_args.cache_dir)['test'].select(range(1000))#1000개 샘플만 테스트

    valid_dataset = ConstantLengthDataset(#미세튜닝 후 테스트할때는 ConstantLengthDataset하지 말고
        tokenizer,
        valid_data,
        formatting_func=prepare_sample_text,
        infinite=False,
        seq_length=args.seq_length,
        chars_per_token=chars_per_token,
    )
    #print(len(train_dataset), len(valid_dataset))#len부분이 조금 이상하게 나옴...??
    return train_dataset, valid_dataset



In [ ]:
# 교수님이 tip ppt #4. 최종 테스트 시 ConstantLengthDataset 금지

def create_datasets(tokenizer, args):
    dataset = load_dataset(args.dataset_name, cache_dir=args.cache_dir)

    # Default to using a bit more than 2500 samples for training
    train_data = dataset["train"].select(range(2100))

    # Default to using exactly 1000 samples for validation during testing
    #valid_data = dataset["test"].select(range(1000))

    chars_per_token = chars_token_ratio(train_data, tokenizer)
    print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

    train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=args.seq_length,
        chars_per_token=chars_per_token,
    )
    valid_dataset = load_dataset(
        script_args.dataset_name,
        cache_dir=script_args.cache_dir)["test"].select(range(1000)) # range(1000)

    print(len(train_dataset), len(valid_dataset))#len부분이 조금 이상하게 나옴...??
    return train_dataset, valid_dataset

In [ ]:
train_data = my_dataset["train"]#id,document,label들 확인 용 체크 코드->실행 할 필요x
print("First few examples:", train_data[:5])
test_data=my_dataset["test"]
print("tests:",test_data[:5])

First few examples: {'id': ['9976970', '3819312', '10265843', '9045019', '6483659'], 'document': ['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다'], 'label': [0, 1, 0, 0, 1]}
tests: {'id': ['6270596', '9274899', '8544678', '6825595', '6723715'], 'document': ['굳 ㅋ', 'GDNTOPCLASSINTHECLUB', '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아', '지루하지는 않은데 완전 막장임... 돈주고 보기에는....', '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??'], 'label': [1, 0, 0, 0, 0]}


# 미세 튜닝용 모델 로딩

max steps를 2000이상으로 설정해야 평균적으로 90%내외의 정확도 accuracy가 나오고 loss값이 0.2보다 작은 수치로 도출되었다. 따라서 step은 최소 2000이상, loss값도 0.2 이하로 나와야 정확도가 90%이상을 나오게 할 수 있음을 알 수 있다.

In [ ]:
script_args = ScriptArguments(
    num_workers=2,
    seq_length=384,
    dataset_name='nsmc',
    model_name='KT-AI/midm-bitext-S-7B-inst-v1',
    # model_name='jangmin/midm-7b-safetensors-only',
    )

In [ ]:
script_args.training_args.logging_steps = 100
script_args.training_args.max_steps = 2100
script_args.training_args.output_dir = '/gdrive/MyDrive/Lectures/2023/nlp/hw-midm-7B-nsmc'
script_args.training_args.run_name = 'hw-midm-7B-nsmc'
script_args.training_args.save_total_limit=3# 최근 3개의 체크포인트만 유지
script_args.training_args.save_strategy="steps"# 지정 스텝마다 저장
script_args.training_args.save_steps=300# 지정 스텝을 300으로 설정

In [ ]:
print(script_args)

ScriptArguments(cache_dir=None, model_name='KT-AI/midm-bitext-S-7B-inst-v1', dataset_name='nsmc', seq_length=384, num_workers=2, training_args=TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

원본인 'KT-AI/midm-bitext-S-7B-inst-v1' 는 *.bin 형태로 모델을 제공한다.
- 코랩에서 CPU 메모리 부족 발생

해결책
- safetensors로 변환한 모델을 업로드 하고 이를 사용하기로 한다.

- base_model에 대해서 먼저 훈련 수행하고 trained로 미세튜닝된 모델 로딩후 테스트를 수행한다.
-- 아래의 AutoModelForCausalLM에 대해서 base로 먼저 수행하고 미세튜닝시 그 한칸 아래의 trained로 수행.

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(#base에 대해 진행(base_model용
    script_args.model_name,
    quantization_config=bnb_config,
    device_map="auto",  # {"": Accelerator().local_process_index},
    #token=True,#use auth token 대신 사용
    #is_trainable=True,
    cache_dir=script_args.cache_dir,
    trust_remote_code=True,
)
base_model.config.use_cache = False

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

configuration_midm.py:   0%|          | 0.00/831 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/KT-AI/midm-bitext-S-7B-inst-v1:
- configuration_midm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_midm.py:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

rotary_position_embedding.py:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
trained_model = AutoModelForCausalLM.from_pretrained(#trained_model용
    script_args.training_args.output_dir,#또는 로라 어댑처 업로드한 모델 허브ID-Violet0203/hw-midm-7B-nsmc
    quantization_config=bnb_config,
    device_map="auto",  # {"": Accelerator().local_process_index},
    trust_remote_code=True,
    token=True,  # use oath token 대신 사용
    cache_dir=script_args.cache_dir,
)
trained_model.config.use_cache = False

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

configuration_midm.py:   0%|          | 0.00/831 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/KT-AI/midm-bitext-S-7B-inst-v1:
- configuration_midm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_midm.py:   0%|          | 0.00/61.1k [00:00<?, ?B/s]

rotary_position_embedding.py:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
base_model

MidmLMHeadModel(
  (transformer): MidmModel(
    (wte): Embedding(72192, 4096)
    (rotary_pos_emb): RotaryEmbedding()
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-31): 32 x MidmBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MidmAttention(
          (c_attn): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (c_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): MidmMLP(
          (c_fc): Linear4bit(in_features=4096, out_features=21760, bias=False)
          (c_proj): Linear4bit(in_features=10880, out_features=4096, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
 

In [ ]:
peft_config = script_args.peft_config#여기 수행

In [ ]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=8, target_modules={'c_attn', 'c_proj', 'c_fc'}, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(#여기 수행(토크나이저 정의)base
    script_args.model_name,
    trust_remote_code=True,
    cache_dir=script_args.cache_dir,
)

if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

tokenizer.add_special_tokens(dict(bos_token='<s>'))
base_model.config.pad_token_id = tokenizer.pad_token_id
base_model.config.bos_token_id = tokenizer.bos_token_id

tokenizer_config.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

midm_bitext_tokenization.py:   0%|          | 0.00/12.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/KT-AI/midm-bitext-S-7B-inst-v1:
- midm_bitext_tokenization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


midm_bitext_tokenizer.model:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(#여기 수행(토크나이저 정의)trained
    script_args.model_name,
    trust_remote_code=True,
    cache_dir=script_args.cache_dir,
)

if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

tokenizer.add_special_tokens(dict(bos_token='<s>'))
trained_model.config.pad_token_id = tokenizer.pad_token_id
trained_model.config.bos_token_id = tokenizer.bos_token_id

tokenizer_config.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

midm_bitext_tokenization.py:   0%|          | 0.00/12.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/KT-AI/midm-bitext-S-7B-inst-v1:
- midm_bitext_tokenization.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


midm_bitext_tokenizer.model:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

In [ ]:
training_args = script_args.training_args#여기 수행


In [ ]:
train_dataset, valid_dataset = create_datasets(tokenizer, script_args)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

100%|██████████| 400/400 [00:00<00:00, 4279.02it/s]
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:548: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


The character to token ratio of the dataset is: 1.56
2100 1000


In [ ]:
print(len(train_dataset),len(valid_dataset))

2100 1000


In [ ]:
# base_model용

trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    eval_dataset=None,
    peft_config=peft_config,#교수님께서는 None으로 바꾸라고 하심...?원래는 peft_config였음
    #peft_config=None,
    packing=script_args.packing,
    max_seq_length=script_args.seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:267: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
base_model

MidmLMHeadModel(
  (transformer): MidmModel(
    (wte): Embedding(72192, 4096)
    (rotary_pos_emb): RotaryEmbedding()
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-31): 32 x MidmBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MidmAttention(
          (c_attn): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=12288, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=12288, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (c_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096,

In [ ]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MidmLMHeadModel(
      (transformer): MidmModel(
        (wte): Embedding(72192, 4096)
        (rotary_pos_emb): RotaryEmbedding()
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-31): 32 x MidmBlock(
            (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attn): MidmAttention(
              (c_attn): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=12288, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear4bit(in_features=

In [ ]:
print_trainable_parameters(base_model)

trainable params: 16744448 || all params: 3821510656 || trainable%: 0.4381630592527648


In [ ]:
base_model.get_memory_footprint()

7795015808

In [ ]:
trainer.model.print_trainable_parameters()

trainable params: 16,744,448 || all params: 7,034,347,520 || trainable%: 0.23803839591934178


midm 모델을 주문 문장 이해에 적용시 특징
- 모델 로딩 과정에서 CPU도 5.1기가, 디스크 42.4기가, GPU 메모리: 7,4 기가

구글 코랩 T-4 GPU: 300스텝 (13:47초 예상)

시퀀스 길이 384의 경우
- 14.7 G / 15.0 G 사용
- 메모리 오버플로우 발생시 이보다 줄일 것

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:570: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


Step,Training Loss
100,1.273700
200,0.760200
300,0.701600
400,0.709000
500,0.699600
600,0.596800
700,0.564500
800,0.521500
900,0.494400
1000,0.482900


/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:570: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:570: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:642: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7ae54f7115d0> was reported to be 2100 (when accessing len(dataloader)), but 2101 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:642: UserWarning: Length of IterableDataset <trl.trainer.utils.ConstantLengthDataset object at 0x7ae54f7115d0> was reported to be 2100 (when accessing len(dataloader)), but 2

TrainOutput(global_step=2100, training_loss=0.45654016222272603, metrics={'train_runtime': 6005.2876, 'train_samples_per_second': 0.699, 'train_steps_per_second': 0.35, 'total_flos': 6.52085594947584e+16, 'train_loss': 0.45654016222272603, 'epoch': 2.0})

In [ ]:
script_args.training_args.output_dir#trainer.train하고 여기 클릭

'/gdrive/MyDrive/Lectures/2023/nlp/hw-midm-7B-nsmc'

In [ ]:
trainer.save_model(script_args.training_args.output_dir)#그다음 여기 클릭

# 추론 테스트

In [ ]:
from transformers import pipeline, TextStreamer

In [ ]:
instruction_prompt_template = """###System;다음은 네이버 영화의 리뷰문장이다.이를 분석하여 리뷰문장이 긍정인지 부정인지를 분류하고자 한다. 리뷰와 긍정,부정 여부를 출력하는 분석결과를 완성해주기를 바란다.

### 리뷰: {0} ### 분석 결과:
"""

prompt_template = """###System;{System}
\n###User;{User}
\n###Midm;"""

default_system_msg = (
    "너는 먼저 사용자가 작성한 리뷰의 긍정 또는 부정여부를 판단해야 한다. 이로부터 첫번째로 리뷰를 출력하고 두번째로 그에 대한 결과를 '긍정' 또는 '부정' 으로 출력해야한다."
)

In [ ]:
evaluation_queries = [
    "굳 ㅋ",
    "카밀라벨 발연기",
    "신들린 연기가 정말 돋보인다",#내가 추가한 문장
    "이런 영화를 영화라고 부를 수는 있나 ㅋㅋㅋㅋㅋ 노잼...",#내가 추가한 문장
    "1%라도 기대했던 내가 죄인입니다 죄인입니다....",
    "뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아",
    "지루하지는 않은데 완전 막장임... 돈주고 보기에는....",
    "3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??",
    "참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다.그냥 까고싶어서 안달난것처럼 보인다"
]


In [ ]:
def wrapper_generate(model, input_prompt, do_stream=False):
    data = tokenizer(input_prompt, return_tensors="pt")
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    input_ids = data.input_ids[..., :-1]
    with torch.no_grad():
        pred = model.generate(
            input_ids=input_ids.cuda(),
            streamer=streamer if do_stream else None,
            use_cache=True,
            max_new_tokens=float('inf'),
            do_sample=False
        )
    decoded_text = tokenizer.batch_decode(pred, skip_special_tokens=True)
    decoded_text = decoded_text[0].replace("<[!newline]>", "\n")
    return (decoded_text[len(input_prompt):])

In [ ]:
eval_dic = {i:wrapper_generate(model=trained_model, input_prompt=prompt_template.format(System=default_system_msg, User=evaluation_queries[i]))for i, query in enumerate(evaluation_queries)}

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [ ]:
print(eval_dic[0])

 긍정


# 미세튜닝된 모델 로딩 후 테스트

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    script_args.training_args.output_dir,
    quantization_config=bnb_config,
    device_map="auto",
    cache_dir=script_args.cache_dir,
    trust_remote_code=True,
)

ValueError: ignored

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    script_args.model_name,
    trust_remote_code=True,
    cache_dir=script_args.cache_dir,
)

if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

tokenizer.add_special_tokens(dict(bos_token='<s>'))

trained_model.config.pad_token_id = tokenizer.pad_token_id
trained_model.config.bos_token_id = tokenizer.bos_token_id

추론 과정에서는 GPU 메모리를 약 5.5 GB 활용

eval_dic에서 자꾸 리뷰는 안나오고 결과인 긍정,부정만 나오는 문제가 있음...

In [ ]:
valid_dataset=valid_dataset.select(range(1000))

In [ ]:
eval_dic = {i:wrapper_generate(model=trained_model, do_stream=True, input_prompt=prompt_template.format(System=default_system_msg, User=valid_dataset[i]))for i, query in enumerate(valid_dataset)}

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


긍정
긍정
부정
부정
부정
긍정
부정
부정
부정
부정
긍정
부정
긍정
긍정
부정
부정
긍정
부정
긍정
부정
긍정
부정
긍정
긍정
부정
긍정
긍정
긍정
부정
긍정
긍정
긍정
부정
긍정
부정
부정
부정
부정
부정
부정
부정
부정
부정
부정
부정
긍정
긍정
부정
긍정
긍정
부정
부정
부정
긍정
긍정
긍정
부정
긍정
긍정
긍정
부정
부정
부정
부정
긍정
부정
긍정
긍정
긍정
긍정
부정
부정
부정
부정
부정
긍정
긍정
부정
긍정
부정
부정
긍정
긍정
긍정
부정
긍정
긍정
부정
긍정
부정
부정
긍정
부정
부정
부정
부정
긍정
긍정
부정
긍정
부정
부정
긍정
부정
긍정
부정
긍정
부정
부정
긍정
부정
긍정
긍정
긍정
긍정
부정
긍정
긍정
긍정
긍정
부정
부정
긍정
긍정
부정
긍정
부정
긍정
부정
긍정
부정
긍정
부정
긍정
긍정
부정
긍정
긍정
긍정
긍정
부정
부정
긍정
긍정
부정
긍정
부정
부정
부정
부정
긍정
부정
긍정
긍정
긍정
긍정
긍정
부정
긍정
부정
긍정
긍정
긍정
긍정
긍정
부정
긍정
긍정
긍정
부정
부정
부정
부정
긍정
긍정
부정
긍정
부정
부정
긍정
부정
긍정
부정
부정
긍정
긍정
부정
부정
부정
긍정
긍정
부정
부정
긍정
긍정
부정
긍정
긍정
부정
부정
부정
긍정
긍정
긍정
긍정
긍정
긍정
부정
부정
긍정
부정
긍정
부정
긍정
부정
부정
부정
긍정
긍정
부정
긍정
긍정
부정
부정
부정
긍정
긍정
부정
긍정
긍정
부정
부정
긍정
긍정
긍정
부정
부정
긍정
부정
부정
부정
부정
긍정
긍정
부정
긍정
긍정
긍정
긍정
부정
부정
부정
긍정
부정
부정
긍정
긍정
긍정
부정
부정
부정
긍정
긍정
긍정
긍정
부정
긍정
부정
긍정
긍정
긍정
부정
부정
긍정
긍정
긍정
부정
긍정
부정
긍정
부정
긍정
긍정
부정
부정
부정
긍정
긍정
부정
긍정
긍정
긍정
긍정
부정
긍정
부정
부정
부정
긍정
긍정
긍정
긍정
긍정
부정
부정
긍정
부정
긍정
긍정
긍정
부정
긍정
긍정
부정
부정
부정
부정
부정
부정
긍정
부정
부정
긍정
긍정
부정
긍정
부정
긍정
긍정
부정
긍정
긍정
부정
부

In [ ]:
correct_predictions = 0#100개 샘플에 대한 정확도는 약 87% 1000개 샘플은 90%부근
total_examples = len(valid_dataset)

for i, query in enumerate(valid_dataset):
    actual_label = int(query["label"])
    predicted_label = 1 if eval_dic[i] == "긍정" else 0  # Assuming "긍정" is positive, "부정" is negative

    if actual_label == predicted_label:
        correct_predictions += 1

accuracy = correct_predictions / total_examples
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 90.10%


In [ ]:
import pandas as pd

data = {'TP': [0, 0], 'TN': [0, 0]}
correct_predictions = 0

for i, query in enumerate(valid_dataset):
    actual_label = int(query["label"])
    predicted_label = 1 if eval_dic[i] == "긍정" else 0

    if predicted_label == 1:
        if actual_label == 1:
            data['TP'][0] += 1  # PP
            correct_predictions += 1
        else:
            data['TP'][1] += 1  # PN
    else:
        if actual_label == 1:
            data['TN'][0] += 1  # PP
        else:
            data['TN'][1] += 1  # PN
            correct_predictions += 1

df = pd.DataFrame(data, index=['PP', 'PN'])

accuracy = correct_predictions / len(valid_dataset)

df.loc['Accuracy'] = ["-", accuracy]

print(df)

           TP       TN
PP        438   70.000
PN         29  463.000
Accuracy    -    0.901


In [ ]:
for i, (query, result) in enumerate(zip(evaluation_queries, eval_dic.values())):
    print(f"- 분석 결과 {i}: 리뷰: {query}, 결과: {result.strip()}")#혹시나 리뷰:결과 이런형태로 출력해야할까봐 시도해본 코드...(내가 짠..)

- 분석 결과 0: 리뷰: 굳 ㅋ, 결과: 긍정
- 분석 결과 1: 리뷰: 카밀라벨 발연기, 결과: 부정
- 분석 결과 2: 리뷰: 신들린 연기가 정말 돋보인다, 결과: 긍정
- 분석 결과 3: 리뷰: 이런 영화를 영화라고 부를 수는 있나 ㅋㅋㅋㅋㅋ 노잼..., 결과: 부정
- 분석 결과 4: 리뷰: 1%라도 기대했던 내가 죄인입니다 죄인입니다...., 결과: 부정
- 분석 결과 5: 리뷰: 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아, 결과: 부정
- 분석 결과 6: 리뷰: 지루하지는 않은데 완전 막장임... 돈주고 보기에는...., 결과: 부정
- 분석 결과 7: 리뷰: 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??, 결과: 부정
- 분석 결과 8: 리뷰: 참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다.그냥 까고싶어서 안달난것처럼 보인다, 결과: 부정


In [ ]:
print(eval_dic[0])#여기 실행

긍정


## 해당 파일을 나의 부계정인 laphoria01@gmail.com에서 수행했기 때문에 create report(wandb)를 수행한 결과 그래프를 html방식으로 제출하였다. 해당 링크를 들어가면 링크를 가진 사용자들은 접근할 수 있도록 권한을 설정했다.(train loss, epoch등 정보 확인 가능)

In [ ]:
urls="https://wandb.ai/laphoria01/huggingface/reports/hw-midm-7b-nsmc--Vmlldzo2MjcwMjA2"
from IPython.display import HTML
wandb_url =urls
html_code = f'<a href="{wandb_url}" target="_blank">Open the WandB Report</a>'
HTML(html_code)